<a href="https://colab.research.google.com/github/gretiere545/corpus/blob/main/corpus_sync_airtable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Corpus Airtable API

In [1]:
#!/usr/bin/env python
# -*- coding: utf8 -*-
!pip install gspread --upgrade
# fix here => https://stackoverflow.com/questions/71347973/modulenotfounderror-no-module-named-gspread-models
!pip uninstall gspread-dataframe
!pip install gspread-dataframe
!pip install gspread-formatting
!pip install airtable-python-wrapper
import pandas as pd
import numpy as np
import requests
import json
import uuid
from datetime import datetime
import datetime
pd.set_option("display.width",1000)
# Ce bout de code pour pouvoir downloader des fichiers sur le drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Trad-Union/Corpus/ASAMLA
from google.colab import files

# Ce bout de code pour pouvoir downloader des google sheets dans des dataframes
from google.colab import auth
auth.authenticate_user()
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from gspread_formatting import *
from oauth2client.client import GoogleCredentials
# fix here => https://github.com/burnash/gspread/issues/1014
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
#gc = gspread.authorize(GoogleCredentials.get_application_default())
from airtable import Airtable

!pip install txtai[pipeline]
!pip install sacremoses
from txtai.pipeline import Translation
translate = Translation()

!export PYTHONIOENCODING=utf8


Found existing installation: gspread-dataframe 3.3.0
Uninstalling gspread-dataframe-3.3.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/gspread_dataframe-3.3.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/gspread_dataframe.py
Proceed (y/n)? y
  Successfully uninstalled gspread-dataframe-3.3.0
  Using cached gspread_dataframe-3.3.0-py2.py3-none-any.whl (7.9 kB)
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Trad-Union/Corpus/ASAMLA


In [2]:
def get_cc_config(f):
  vk = []
  try:
    with open(f) as vk_f:
        vk = json.load(vk_f)
  except BaseException as e:
    print(e)
  return vk

def set_cc_config (vk, f):
  # sauvegarde du fichier de paramétrage
  with open(f, 'w') as fp:
      json.dump(vk, fp)  

In [3]:
#
# Ouverture de la Sheet Centrale (corpus_central_base)
#
def get_CCDB_wb(uri):
  sheet_central = uri
  wb_central = gc.open_by_url(sheet_central)
  return wb_central

def get_CCDB_data(wb, idx):
  t_corpus = wb.get_worksheet(idx)
  data_t_corpus = t_corpus.get_all_values()
  return data_t_corpus

#
# DataFrame CC global (toutes les langues)
#
def get_ccdf_global(CCDB):
  return pd.DataFrame(CCDB[1:], columns=CCDB[0])

#
# DataFrame CC Français (invariable)
#
def get_ccdf_fr(df_cc):
  # on supprime les doublons
  return df_cc[['uid','expression','theme','état','date','commentaires','index']]

In [4]:
# Config Langues
config_languages = 'med_vac_synthese.json'
vk_languages = get_cc_config(config_languages)

In [5]:
# Ouverture de la Sheet Centrale
sheet = 'https://docs.google.com/spreadsheets/d/1j9mtvKJn0Ad3FqxoJXMSKLJOfYTDOhtw3wxXZ4sQAIc'
wb  = gc.open_by_url(sheet)

In [6]:
# Config Corpus (de consolidation)
config_corpus_central = "corpus_central_config.json"
vk_corpus_central = get_cc_config(config_corpus_central)

In [7]:
# Base Centrale
# on charge l'onglet 1 (français)
db_cc_wb = get_CCDB_wb(vk_corpus_central[0]["uri"])
db_cc = get_CCDB_data (db_cc_wb, 1)
df_cc_global = get_ccdf_global(db_cc)
df_cc_fr = get_ccdf_fr(df_cc_global).drop_duplicates()

# Comparaison des versions de référentiels

In [8]:
def getLastModifiedTime (wb):
  # https://stackoverflow.com/questions/64254039/python-how-to-check-if-a-google-spreadsheets-was-updated
  # https://stackoverflow.com/questions/52260789/update-googlesheet-cell-with-timestamp-from-python
  revisions_uri = f'https://www.googleapis.com/drive/v3/files/{wb.id}/revisions'
  headers = {'Authorization': f'Bearer {GoogleCredentials.get_application_default().get_access_token().access_token}'}
  response = requests.get(revisions_uri, headers=headers).json()
  return response['revisions'][-1]['modifiedTime']

In [9]:
print (getLastModifiedTime(db_cc_wb))

2022-04-01T12:52:39.368Z


In [10]:
df_cc_global.loc[df_cc_global["expression"]=="Anesthésie générale"]

,uid,expression,date,theme,état,commentaires,index,ams,eng,tur,...,arm,dar,pst,prs,aze,esp,amh,all,pol,som
174,ef2f771b,Anesthésie générale,,Médecine générale,,,A,تخدير عام,,Genel anestezi,...,,,,,,,,,Znieczulenie ogólne,


### Appel de l'API
https://towardsdatascience.com/downloading-airtable-data-into-python-89e5c7107a24

In [11]:
params = ()
api_key = "keyrI98TIqu6mbFcf" 
api_key_gmail = "keysgpJWq3aWwqSZe"
headers = {"Authorization": "Bearer " + api_key,"Content-Type" : "application/json"}
base_id = "appawT3gg7cJhPYv0" # Base Nouvelle HV
table_name = 'Corpus'

In [12]:
airtable = Airtable(base_id, table_name, api_key)

```
for page in airtable.get_iter(view='Grid view',sort='Proposition'):
  for record in page:
    value = record['fields']['Proposition']
```



In [13]:
def airtable_fetch_all_records():
  vk_at = airtable.get_all(view='Admin',sort='Proposition')
  df_vkat = pd.DataFrame.from_records((r['fields'] for r in vk_at))
  return df_vkat

In [14]:
def getNotInGSheet(df_cc_global, df_vkat):
  df_not_in_gsheet = df_vkat[~df_vkat["UID"].isin(df_cc_global["uid"])]
  return df_not_in_gsheet

In [15]:
df_vkat = airtable_fetch_all_records()

In [16]:
master="langdeck"

In [17]:
def txtai_alpha (input, iso_code):
  try:
    translation = translate(input, iso_code)
  except Exception as e:
    translation = ""
    pass
  return translation, 2

In [18]:
def fetch_pivot_en (txt_fr):
  try:
    translation = translate(txt_fr, "en")
    print (translation)
  except Exception as e:
    translation = ""
    pass  
  return translation

## on met à jour dans Airtable

In [19]:
'''
df_vkat_pivot = df_vkat.copy()
df_vkat["pivot_en"] = df_vkat[["Pivot","eng"]].apply(lambda x:x[1] if not pd.isna(x[1]) else fetch_pivot_en(x[0]), axis=1)
if master == "langdeck":
  for index, row in df_vkat_pivot.iterrows():
    if row["UID"] is not None:
      record = airtable.match('UID', row["UID"])
      if bool(record):
        aDict = {}
        key = "UID"
        aDict[key] = row["UID"]
        aDict["pivot_en"] = row["pivot_en"]
        airtable.update(record['id'], aDict)
'''

'\ndf_vkat_pivot = df_vkat.copy()\ndf_vkat["pivot_en"] = df_vkat[["Pivot","eng"]].apply(lambda x:x[1] if not pd.isna(x[1]) else fetch_pivot_en(x[0]), axis=1)\nif master == "langdeck":\n  for index, row in df_vkat_pivot.iterrows():\n    if row["UID"] is not None:\n      record = airtable.match(\'UID\', row["UID"])\n      if bool(record):\n        aDict = {}\n        key = "UID"\n        aDict[key] = row["UID"]\n        aDict["pivot_en"] = row["pivot_en"]\n        airtable.update(record[\'id\'], aDict)\n'

## Synchronisation du serveur vers airtable

### 1. Différence entre A et B (sur UID)

In [20]:
df_not_in_airtable = df_cc_global[~df_cc_global["uid"].isin(df_vkat["UID"])].dropna()
df_not_in_airtable

,uid,expression,date,theme,état,commentaires,index,ams,eng,tur,...,arm,dar,pst,prs,aze,esp,amh,all,pol,som


### On ajoute les éléments de A absents dans B

In [21]:
if master != "langdeck":
  for index, row in df_not_in_airtable.iterrows():
    if row["uid"] is not None:
      record = airtable.match('UID', row["uid"])
      if bool(record) is False:
        aDict = {}
        key = "UID"
        aDict[key] = row["uid"]
        aDict["Proposition"] = row["expression"]
        aDict["idx"] = row["index"]
        for i in vk_languages:
          key = i["trigramme"]
          aDict[key] = row[key]
        airtable.insert(aDict)

### On met à jour les traductions de A présents dans B

> Bloc en retrait



In [22]:
if master != "langdeck":
  cpt = 0
  max = len(df_cc_global) 
  for index, row in df_cc_global.iterrows():
    if row["uid"] is not None:
      record = airtable.match('UID', row["uid"])
      if bool(record):
        aDict = {}
        key = "UID"
        aDict[key] = row["uid"]
        for i in vk_languages:
          key = i["trigramme"]
          aDict[key] = row[key]
        airtable.update(record['id'], aDict)
        cpt+=1
        print(str(cpt/max) + " %", end="\r", flush=True)      
        #print (record['id'] +" " + row["uid"] +" " + row["tur"])

### On récupère les éléments de B absents de A

In [23]:
df_not_in_gsheet = getNotInGSheet(df_cc_global, df_vkat)
#df_not_in_gsheet[["Proposition","idx_cor","UID"]]

## Mise en forme et ajustements dans Airtable

In [24]:
def make_unique_id():
  return str(uuid.uuid4())[:8]

In [25]:
df_vkat = airtable_fetch_all_records()

### Update des UIDS et des HASH manquants

In [26]:
def chk_uid (vk_uid, record):
  try:
    value = record['fields']['UID']
  except KeyError:
    # uid manquant
    print ("mssing uid")
    while True:
      auid = make_unique_id()
      if auid not in vk_uid:
        aDict = {}
        aDict["UID"] = auid
        print (record['id'] + " " + aDict["UID"])
        airtable.update(record['id'], aDict)
        vk_uid.append(auid) 
        break 
  finally:
    return

In [27]:
def airtable_check_missing_uids():
  vk_uid = df_vkat['UID'].tolist()
  for page in airtable.get_iter(view='Admin',sort='Proposition'):
    for record in page:
      chk_uid (vk_uid, record)


In [28]:
airtable_check_missing_uids()

## Toggle des enregistrements de AT à ajouter dans la sheet locale

In [29]:
if master != "langdeck":
  df_vkat = airtable_fetch_all_records()
  df_not_in_gsheet = getNotInGSheet(df_cc_global, df_vkat)
  # on met à jour le status dans Airtable
  for index, row in df_not_in_gsheet.iterrows():
    if row["UID"] is not None:
      record = airtable.match('UID', row["UID"])
      if bool(record) is True:
        aDict = {}
        aDict["AppendToCorpus"] = True
        airtable.update(record['id'], aDict)
        print ("Updating status for " + row["UID"])

# Report dans la sheet locale

In [30]:
if master != "langdeck":
  # Ouverture de la Sheet Locale (mots à traduire)
  sheet = 'https://docs.google.com/spreadsheets/d/1M3CFSpAGII25qCl0apYrRPX-DmpdxDFLLB_Ob-MX8DQ/'
  wb  = gc.open_by_url(sheet)
  t_corpus = wb.worksheet("fra_med_gen")
  data_t_corpus = t_corpus.get_all_values()
  df_langdeck = pd.DataFrame(data_t_corpus[1:], columns=data_t_corpus[0]).drop_duplicates()  
  df_not_in_gsheet.rename(columns={'UID': 'uid', 'Proposition': 'français'}, inplace=True)
  df_not_in_gsheet = df_not_in_gsheet[["uid","français"]]
  frames = [df_langdeck, df_not_in_gsheet]
  result = pd.concat(frames)
  result.drop_duplicates(subset=['uid'], inplace=True, keep='last')
  set_with_dataframe(t_corpus, result)

# Regroupement pour Langdeck

In [31]:
def compare_langdeck_time(hash, time):
  ts1 = time
  ts2 = df_langdeck.loc[df_langdeck["hash"]==hash]["unixtime"].values[0]
  return ts1 > ts2


In [32]:
def compare_airtable_time(hash, time):
  ts1 = time
  ts2 = df_airtable_for_langdeck.loc[df_airtable_for_langdeck["hash"]==hash]["unixtime"].values[0]
  return ts1 > ts2

In [33]:
def get_df_airtable_for_langdeck():
  df_vkat = airtable_fetch_all_records()
  df_temp = df_vkat.copy()
  df_temp = df_temp[["UID","Maitre","Ordre","Proposition","pivot_en","idx_cor","Thématique","lastModified","ams","eng","tur","rus","ukr","rou","hun","alb","geo","arm","dar","pst","prs","esp","pol","tig"]]
  df_temp.rename(columns={"UID":"uid","Proposition":"expression","pivot_en":"pivot","idx_cor":"idx", "Maitre":"maitre","Ordre":"ordre","Thématique":"themes","lastModified":"timestamp"}, inplace=True)
  # https://stackoverflow.com/questions/28654047/convert-columns-into-rows-with-pandas
  df_airtable_for_langdeck = pd.melt(df_temp, id_vars=["uid", "maitre","ordre","expression","pivot","idx","timestamp","themes"], 
                    var_name="language", value_name="translation")
  #https://stackoverflow.com/questions/60393668/pandas-generate-unique-id-based-on-row-values
  #https://stackoverflow.com/questions/64604408/transform-dataframe-column-with-a-hash-value
  df_airtable_for_langdeck['hash'] = df_airtable_for_langdeck[['uid', 'language']].sum(axis=1).astype(str)
  #Fonction lambda pour récupérer le nom long de la langue (https://stackoverflow.com/questions/2191699/find-an-element-in-a-list-of-tuples)
  df_airtable_for_langdeck["language_fulltext"] = df_airtable_for_langdeck["language"].apply(lambda x:[item for item in [(a_dict["trigramme"],a_dict["language"]) for a_dict in vk_languages] if item[0] == x][0][1])  
  df_airtable_for_langdeck['timestamp'] = df_airtable_for_langdeck['timestamp'].astype('datetime64[s]')
  df_airtable_for_langdeck['unixtime'] = df_airtable_for_langdeck['timestamp'].astype('datetime64[s]').astype('int')
  # ajout des thématiques  
  # on éclate la liste des thèmes en 3 colonnes
  df_airtable_for_langdeck[["theme_1","theme_2","theme_3"]] = pd.DataFrame([pd.Series(x) for x in df_airtable_for_langdeck.themes])
  return df_airtable_for_langdeck

### Vérification qu'il n'existe pas des doublons (hash)

In [34]:
df_airtable_for_langdeck = get_df_airtable_for_langdeck()
df_airtable_for_langdeck[df_airtable_for_langdeck['hash'].duplicated()].sort_values("hash")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  from ipykernel import kernelapp as app


,uid,maitre,ordre,expression,pivot,idx,timestamp,themes,language,translation,hash,language_fulltext,unixtime,theme_1,theme_2,theme_3


In [35]:
# on éclate la liste des thèmes en 3 colonnes
df_airtable_for_langdeck[["theme_1","theme_2","theme_3"]] = pd.DataFrame([pd.Series(x) for x in df_airtable_for_langdeck.themes])


In [36]:
# on ajoute une colonne de translation state (0=VOID; 1=PENDING; 2=OK; 3=KO)
df_airtable_for_langdeck["translation_state"] = df_airtable_for_langdeck["translation"].apply(lambda x:0 if pd.isna(x) else 1)

## Base Langdeck

In [37]:
db_langdeck_url = "https://docs.google.com/spreadsheets/d/1bkL0bK2yh6Ea_BQaFIJ_yAvbTXw8NOFtlTZJSZK_5FY/" #polyglot-central
db_langdeck_url = "https://docs.google.com/spreadsheets/d/1iJLt3qAKq576eNirMwcsdRltg7nerwa9N70dA0vraa8/" #db-langdeck

In [38]:
def get_df_langdeck(url):
  # Ouverture de la Sheet Langdeck (mots à traduire)

  #sheet = 'https://docs.google.com/spreadsheets/d/1cSnSnnQunL-I7hpyYkIqbtuGr55unVJ3Qe8URSrjTr8/'
  #t_corpus = wb.worksheet("Langdeck")
  sheet = url
  # 'https://docs.google.com/spreadsheets/d/1bkL0bK2yh6Ea_BQaFIJ_yAvbTXw8NOFtlTZJSZK_5FY/'
  # sheet = 'https://docs.google.com/spreadsheets/d/1iJLt3qAKq576eNirMwcsdRltg7nerwa9N70dA0vraa8/'
  wb  = gc.open_by_url(sheet)
  t_corpus = wb.worksheet("t_sync_voc")
  data_t_corpus = t_corpus.get_all_values()
  df_langdeck = pd.DataFrame(data_t_corpus[1:], columns=data_t_corpus[0]).drop_duplicates()  
  df_langdeck['ordre'] = pd.to_numeric(df_langdeck['ordre'])
  df_langdeck['timestamp'] = df_langdeck['timestamp'].astype('datetime64[s]')
  df_langdeck['unixtime'] = df_langdeck['timestamp'].astype('datetime64[s]').astype('int')
 
  return df_langdeck

In [39]:
def is_audio(f):
  path = "/content/drive/My Drive/appsheet/data/langdeck-1629444/"
  f = path + f
  result = True
  try:
    file = open(f, "r")
  except FileNotFoundError as e:
    # do nothing
    result = False

  return result

In [40]:
 df_langdeck = get_df_langdeck(db_langdeck_url)
 df_langdeck['is_audio'] = df_langdeck['audio'].apply(lambda x:is_audio(x))
 # df_langdeck.loc[df_langdeck['is_audio']==True]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  from ipykernel import kernelapp as app


## MAJ du status isAudio

In [41]:
# NE PAS UTILISER
'''
df_isaudio = df_langdeck.loc[df_langdeck['is_audio']==True]
sheet = db_langdeck_url
wb  = gc.open_by_url(sheet)
t_corpus = wb.worksheet("t_sync_voc")
for index, row in df_isaudio.iterrows():
  cell = t_corpus.find(row['hash'])
  t_corpus.update_cell(cell.row, 12, row['is_audio'])
  print(index)
'''

'\ndf_isaudio = df_langdeck.loc[df_langdeck[\'is_audio\']==True]\nsheet = db_langdeck_url\nwb  = gc.open_by_url(sheet)\nt_corpus = wb.worksheet("t_sync_voc")\nfor index, row in df_isaudio.iterrows():\n  cell = t_corpus.find(row[\'hash\'])\n  t_corpus.update_cell(cell.row, 12, row[\'is_audio\'])\n  print(index)\n'

## Comparaison 1 : new records dans Airtable à ajouter dans Langdeck

In [42]:
sheet = db_langdeck_url
wb  = gc.open_by_url(sheet)
t_corpus = wb.worksheet("t_sync_voc")

In [43]:
# la différence se fait sur le hash et non l'uid
df_new_in_airtable = df_airtable_for_langdeck[~df_airtable_for_langdeck["hash"].isin(df_langdeck["hash"])]
print (str(len(df_new_in_airtable)) + " new airtable rows detected.")
for index, row in df_new_in_airtable.iterrows():

  if pd.isna(row['theme_1']):
    row['theme_1'] = ""
  if pd.isna(row['theme_2']):
    row['theme_2'] = ""
  if pd.isna(row['theme_3']):
    row['theme_3'] = ""        
  print (row['hash'])    
  t_corpus.append_row([row['hash'],
                       row['uid'], 
                       row['maitre'], 
                       row['ordre'],
                       row['expression'], 
                       row['idx'], 
                       str(pd.to_datetime(row['timestamp'])), 
                       row['language'],                     
                       "", 
                       row['language_fulltext'],
                       "Files_Files_/Corpus/t_expo_audio/" + row['uid'] + "-" + row['language'] + ".mp3",
                       False,
                       "",
                       row['theme_1'],
                       row['theme_2'],
                       row['theme_3']                     
                       ])

# on recharge avec les nouvelles données
df_langdeck = get_df_langdeck(db_langdeck_url)

0 new airtable rows detected.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  from ipykernel import kernelapp as app


In [44]:
len(df_langdeck)

5984

## Comparaison 2 : updated records from Airtable to Langdeck
On met à jour les méta mais pas les traductions (traduction dans langdeck uniq.)

In [45]:
df_airtable_for_langdeck = get_df_airtable_for_langdeck()
df_airtable_for_langdeck[df_airtable_for_langdeck['hash'].duplicated()].sort_values("hash")
# df_airtable_for_langdeck

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  from ipykernel import kernelapp as app


,uid,maitre,ordre,expression,pivot,idx,timestamp,themes,language,translation,hash,language_fulltext,unixtime,theme_1,theme_2,theme_3


Mise à jour (INIT) des thématiques

In [46]:
for index, row in df_airtable_for_langdeck.iterrows():
  # clé A et B : concat des clés primaires
  t = df_langdeck.loc[df_langdeck["hash"]==row['hash']].applymap(str)
  # key_b = langdeck (sheet)
  try:
    key_b = t["theme_1"].item() + t["theme_2"].item() + t["theme_3"].item()
  except Exception as e:
    print (row['hash'] + str(e))
    pass

  if not pd.isna(row['theme_1']):
    key_a = row['theme_1']
  else:
    row['theme_1'] = ""
    key_a = row['theme_1']

  if not pd.isna(row['theme_2']): 
    key_a = key_a + row['theme_2']  
  else:
    row['theme_2'] = ""
    key_a = key_a + row['theme_2']        
      
  if not pd.isna(row['theme_3']):
    key_a = key_a + row['theme_3']  
  else:
    row['theme_3'] = ""    
    key_a = key_a + row['theme_3'] 

  # si lex clés diffenet
  
  if key_a != key_b:  
    cell = t_corpus.find(row['hash'])
    print ("Updating cell " + str(cell.row))
    t_corpus.update_cell(cell.row, 14, row['theme_1'])
    t_corpus.update_cell(cell.row, 15, row['theme_2'])
    t_corpus.update_cell(cell.row, 16, row['theme_3'])

In [47]:
df_langdeck = get_df_langdeck(db_langdeck_url)
df_airtable_for_langdeck["last_updated"] = df_airtable_for_langdeck[["hash","unixtime"]].apply(lambda x:compare_langdeck_time(x[0],x[1]), axis=1) 
df_newly_updated_in_airtable = df_airtable_for_langdeck.loc[df_airtable_for_langdeck["last_updated"]]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  from ipykernel import kernelapp as app


In [89]:
sheet = db_langdeck_url
wb  = gc.open_by_url(sheet)
t_corpus = wb.worksheet("t_sync_voc")
# on recharge avec les nouvelles données
df_langdeck = get_df_langdeck(db_langdeck_url)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  from ipykernel import kernelapp as app


In [49]:
for index, row in df_newly_updated_in_airtable.iterrows():
  # clé A et B : concat des clés primaires
  key_a = row['maitre']+str(row["ordre"])+row["expression"]+row["idx"]
  t = df_langdeck.loc[df_langdeck["hash"]==row['hash']].applymap(str)
  key_b = t["maitre"].item() + t["ordre"].item() + t["expression"].item() + t["idx"].item()
  # si lex clés diffenet
  if key_a != key_b:
    cell = t_corpus.find(row['hash'])
    t_corpus.update_cell(cell.row, 3, row['maitre'])
    t_corpus.update_cell(cell.row, 4, row['ordre'])  
    t_corpus.update_cell(cell.row, 5, row['expression'])  
    t_corpus.update_cell(cell.row, 6, row['idx'])
    t_corpus.update_cell(cell.row, 7, str(pd.to_datetime(row['timestamp'])))    
    print ("Update from Airtable Meta " + key_a)
# on recharge avec les nouvelles données
df_langdeck = get_df_langdeck(db_langdeck_url)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  from ipykernel import kernelapp as app


## Comparaison 3 : updated records from Langdeck to Airtable

In [50]:
df_langdeck["last_updated"] = df_langdeck[["hash","unixtime"]].apply(lambda x:compare_airtable_time(x[0],x[1]), axis=1) 
df_newly_updated_in_langdeck = df_langdeck.loc[df_langdeck["last_updated"]]

In [51]:
df_airtable_for_langdeck.dtypes

uid                          object
maitre                       object
ordre                         int64
expression                   object
pivot                        object
idx                          object
timestamp            datetime64[ns]
themes                       object
language                     object
translation                  object
hash                         object
language_fulltext            object
unixtime                      int64
theme_1                      object
theme_2                      object
theme_3                      object
last_updated                   bool
dtype: object

In [52]:
df_newly_updated_in_langdeck

,hash,uid,maitre,ordre,expression,idx,timestamp,language,translation,language_fulltext,...,uploaded_file,theme_1,theme_2,theme_3,theme_1_svg,theme_2_svg,theme_3_svg,unixtime,translation_state,last_updated


## Mise à jour dans Airtable + maj symétrique du timestamp dans Langdeck

In [53]:
for index, row in df_newly_updated_in_langdeck.iterrows():
  if row["uid"] is not None:
    record = airtable.match('UID', row["uid"])
    if bool(record) is True:
      aDict = {}
      aDict[row["language"]] = row["translation"]
      airtable.update(record['id'], aDict)
      print ("Updating status for " + row["uid"])


In [54]:
for index, row in df_newly_updated_in_langdeck.iterrows():
  if row["uid"] is not None:
    record = airtable.match('UID', row["uid"])
    if bool(record) is True:
      # mise à jour symétrique pour garantir le même timestamp
      cell = t_corpus.find(row['hash'])
      t_corpus.update_cell(cell.row, 6, row["idx"])
      t_corpus.update_cell(cell.row, 7, str(pd.to_datetime(row['timestamp'])))
# on recharge avec les nouvelles données
df_langdeck = get_df_langdeck(db_langdeck_url)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  from ipykernel import kernelapp as app


In [90]:
# on ajoute une colonne de translation state (0=VOID; 1=PENDING; 2=OK; 3=KO)
#df_langdeck["translation_state"] = df_langdeck["translation"].apply(lambda x:0 if x=="" else 1)
df_langdeck = get_df_langdeck(db_langdeck_url)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  from ipykernel import kernelapp as app


In [91]:
def get_translation_pivot (hash):
  r = df_airtable_for_langdeck.loc[df_airtable_for_langdeck["hash"]==hash]["pivot"].values.item()
  return r

In [92]:
# on ajoute une colonne pour la translation pivot (en)
df_langdeck["translation_pivot"] = df_langdeck["hash"].apply(lambda x:get_translation_pivot(x))

In [97]:
df_langdeck["translation_state"] = df_langdeck["translation_state"].astype(int)
df_langdeck["translation_ai_source"] = np.nan

In [95]:
df_langdeck.dtypes

hash                             object
uid                              object
maitre                           object
ordre                             int64
expression                       object
idx                              object
timestamp                datetime64[ns]
language                         object
translation                      object
language_fulltext                object
audio                            object
is_audio                         object
uploaded_file                    object
theme_1                          object
theme_2                          object
theme_3                          object
theme_1_svg                      object
theme_2_svg                      object
theme_3_svg                      object
unixtime                          int64
translation_state                 int64
translation_pivot                object
translation_ai_source           float64
dtype: object

In [98]:
df_langdeck.iloc[0]

hash                                                           f9bc13efams
uid                                                               f9bc13ef
maitre                                                          Médicament
ordre                                                                    3
expression               À quel moment de la journée vous prenez le méd...
idx                                                                      M
timestamp                                              2022-04-01 08:15:21
language                                                               ams
translation                              في أي وقت من اليوم تتناول الدواء؟
language_fulltext                                                    Arabe
audio                    Files_Files_/Corpus/t_expo_audio/f9bc13ef-ams.mp3
is_audio                                                             FALSE
uploaded_file                                                             
theme_1                  

# On cherche une traduction avec l'IA si manquante

In [100]:
def fetch_translation_ia_txtai (input, language, filter, ai_code):

  iso_codes = {"ams": "ar",
              "rus": "ru",
              "ukr": "uk",
              "rou": "ro",
              "alb": "sq",
              "eng": "en",
              "tur": "tr",
              "tig": "ti",
              "geo": "ka",
              "arm": "hy",
              "dar": "fa",
              "prs": "fa",
              "pst": "ps",
              "esp": "es",
              "pol": "pl",
              "som": "so",
              "amh": "am",
              "all": "de",
              "hun": "hu" }  

  iso_code = iso_codes[language]
  translation = ""
  state = 0 #par défaut
  if language == filter:
    try:
      # print (input + " : " + language)
      translation = translate(input, iso_code)
      if translation != "":
        print (input + " : " + translation)
        state = 2 #found
    except Exception as e:
      translation = ""
      state = 0
      pass

  return translation, state, ai_code

In [ ]:
df_langdeck.iloc[1367]

In [103]:
df_langdeck.loc[df_langdeck["translation_state"]==2]

,hash,uid,maitre,ordre,expression,idx,timestamp,language,translation,language_fulltext,...,theme_1,theme_2,theme_3,theme_1_svg,theme_2_svg,theme_3_svg,unixtime,translation_state,translation_pivot,translation_ai_source
7,d1746bc6ams,d1746bc6,Accoucher,1,Accoucher,A,2022-03-31 14:33:23,ams,أن تلد الولادة,Arabe,...,Médecine générale,,,Files_Files_/misc/Médical.png,,,1648737203000000000,2,To give birth,11
25,8e7e564dams,8e7e564d,Digestif,2,Appareil digestif,D,2022-04-01 08:25:13,ams,الجهاز المعوي المعوي,Arabe,...,Médecine générale,,,Files_Files_/misc/Médical.png,,,1648801513000000000,2,Gastrointestinal system,11
26,dd4e9883ams,dd4e9883,Assurance,1,Assurance,A,2022-03-31 14:33:25,ams,المؤن (الركان),Arabe,...,Médecine générale,,,Files_Files_/misc/Médical.png,,,1648737205000000000,2,Assurance,11
29,57316191ams,57316191,Avant-bras,1,Avant-bras,A,2022-03-31 15:25:17,ams,Avant-bras,Arabe,...,Corps humain,,,"data:image/svg+xml;utf8,<svg xmlns=""http://www...",,,1648740317000000000,2,Avant-bras,11
37,fff52418ams,fff52418,Allergie,3,Avez-vous des allergies aux médicaments ?,A,2022-04-01 08:22:41,ams,هل لديك أي حساسية للأدوية؟,Arabe,...,Médecine générale,,,"data:image/svg+xml;utf8,<svg xmlns=""http://www...",,,1648801361000000000,2,Do you have any allergies to medication?,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5528,056fd2f2ams,056fd2f2,Profession,3,Quelle profession exercez-vous ?,P,2022-05-10 08:01:05,ams,ما المهنة التي تمارسها؟,Arabe,...,Dossier du patient,Etat civil,,,,,1652169665000000000,2,What profession do you practice?,11
5529,e352c9c7ams,e352c9c7,Sinusite,1,Sinusite,S,2022-05-10 08:01:05,ams,أضف تعليقًا,Arabe,...,Médecine générale,Maladie,,,,,1652169665000000000,2,Sine [K],11
5531,7019983eams,7019983e,Ulcère,2,Ulcère de l'estomac,U,2022-05-10 08:01:06,ams,مُلْسَكَر,Arabe,...,Hépato-Gastroentérologie,Maladie,,,,,1652169666000000000,2,Stomach ulcer,11
5532,a2e70804ams,a2e70804,Ulcère,2,Ulcère de l'intestin grêle,U,2022-05-10 08:01:06,ams,أُجُزأة صغيرة من الأم,Arabe,...,Hépato-Gastroentérologie,Maladie,,,,,1652169666000000000,2,Ulcer of the small intestine,11


In [105]:
filter = "ukr"
ai = 11
# algorithme :
# si state==1(pending) ou 2(validé) => pas d'appel IA
# si state==0 => appel ia
df_langdeck[["translation","translation_state","translation_ai_source"]] = df_langdeck[["translation","translation_pivot","language","translation_state"]].apply(
    lambda x:fetch_translation_ia_txtai(x[1],x[2], filter, ai) 
    if x[3]==0 
    else (x[0],x[3],ai), 
    axis=1, 
    result_type="expand")

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/790k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/984k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.26M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/831k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.98M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.44M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Plier : Травень
Fold your legs! : Загніть ноги!
Occupation : Професія
What is the reason for the consultation? : Яка ж причина на цю пораду?
What profession do you practice? : Яку професію ви займаєте?
Sine [K] : Створення [K]
Ulcer : Ulcer
Stomach ulcer : Виразка шлунка
Ulcer of the small intestine : Ульсер тонкої кишки
Go for it. : Давай.


In [106]:
try:
  t_sync_voc = wb.worksheet("t_sync_voc")
  wb.del_worksheet(t_sync_voc)
except:
  print ("Onglet inexistant !")

wb.add_worksheet("t_sync_voc", 1, 1)
export_sheet = wb.worksheet("t_sync_voc")
set_with_dataframe(export_sheet, df_langdeck)

# Vue Lexique

In [107]:
t_lexique = wb.worksheet("t_lex_theme")
# on recharge avec les nouvelles données

### nouvelles colonnes Thèmes

In [108]:
# on crée une liste des valeurs uniques des 3 colonnes de thèmes
frames = [df_langdeck["theme_1"],df_langdeck["theme_2"],df_langdeck["theme_3"]]
vk_theme = list(set(pd.concat(frames).to_list()))
vk_theme.remove('')

In [109]:
# on ajoute les colonnes à partir de la liste et on valorise T/F
df_lexique = df_langdeck.copy()
df_lexique = pd.concat([df_lexique, pd.DataFrame(columns = vk_theme)])
for item in vk_theme:
  print (item)
  df_lexique[item] = df_lexique[["theme_1","theme_2","theme_3"]].apply(lambda x:True if (x[0]==item or x[1]==item or x[2]==item) else False, axis=1)

Médecine générale
Corps humain
Dossier du patient
Vaccination
Etat civil
Maladie
Hépato-Gastroentérologie
Douleur
Examen du patient


In [110]:
frames=[]
df=pd.DataFrame()
frames.append(df)
# pour chaque thème on ajoute les expressions trouvées
for item in vk_theme:
  df_temp = df_lexique.loc[df_lexique[item] == True]
  df_temp.insert(0,'theme', item)
  df_temp.insert(0,'section', df_temp["ordre"].apply(lambda x:("Vocabulaire" if x==1 else "Expression" if x==2 else "Phrase" if x>=3 else "NONE")))
  df_temp.insert(0,'theme_hash', df_temp["hash"].apply(lambda x:"".join(item.split())+x))
  frames.append(df_temp)
  # on concatène le tout dans un new dataframe
  df_langdeck_theme = pd.concat(frames)

In [87]:
df_langdeck_theme.iloc[0]

theme_hash                                        Médecinegénéralef9bc13efams
section                                                                Phrase
theme                                                       Médecine générale
hash                                                              f9bc13efams
uid                                                                  f9bc13ef
maitre                                                             Médicament
ordre                                                                     3.0
expression                  À quel moment de la journée vous prenez le méd...
idx                                                                         M
timestamp                                                 2022-04-01 08:15:21
language                                                                  ams
translation                                 في أي وقت من اليوم تتناول الدواء؟
language_fulltext                                               

In [111]:
try:
  t_lexique = wb.worksheet("t_lex_theme")
  wb.del_worksheet(t_lexique)
except:
  print ("Onglet inexistant !")

wb.add_worksheet("t_lex_theme", 1, 1)
export_sheet = wb.worksheet("t_lex_theme")
set_with_dataframe(export_sheet, df_langdeck_theme)

# FIN pour l'instant

# Appel IA Google Translate

In [ ]:
translation = translate("Forearm", "ar")
translation

'الكتاب'

In [ ]:
def txtai_alpha (input, iso_code):
  try:
    translation = translate(input, "ar")
  except Exception as e:
    translation = ""
    pass
  return translation, 2


In [ ]:
df_txtai = df_langdeck.loc[(df_langdeck["language"]=="ams") & (df_langdeck["translation_state"]==0) & (df_langdeck["idx"]=="B")]

In [ ]:
df_txtai

,hash,uid,maitre,ordre,expression,idx,timestamp,language,translation,language_fulltext,...,translation_state,Corps humain,Etat civil,Hépato-Gastroentérologie,Examen du patient,Vaccination,Maladie,Dossier du patient,Douleur,Médecine générale
56,f3bff76cams,f3bff76c,Bonjour,1.0,Bonjour,B,2022-03-31 14:33:26,ams,,Arabe,...,0.0,False,False,False,False,False,False,True,False,True
58,b80ffb21ams,b80ffb21,Bouche,1.0,Bouche,B,2022-03-31 15:25:18,ams,,Arabe,...,0.0,True,False,False,False,False,False,False,False,False


In [ ]:
df_txtai[["translation","translation_state"]]=df_txtai[["expression","translation_state"]].apply(lambda x: txtai_alpha(x[0], "ar") if x[1]==0 else "", axis=1, result_type="expand")

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:
df_txtai

,hash,uid,maitre,ordre,expression,idx,timestamp,language,translation,language_fulltext,...,translation_state,Corps humain,Etat civil,Hépato-Gastroentérologie,Examen du patient,Vaccination,Maladie,Dossier du patient,Douleur,Médecine générale
56,f3bff76cams,f3bff76c,Bonjour,1.0,Bonjour,B,2022-03-31 14:33:26,ams,مرحباً، مرحباً، مرحباً، مرحباً، مرحباً، مرحباً...,Arabe,...,2,False,False,False,False,False,False,True,False,True
58,b80ffb21ams,b80ffb21,Bouche,1.0,Bouche,B,2022-03-31 15:25:18,ams,لاشتششششششششششششش,Arabe,...,2,True,False,False,False,False,False,False,False,False


## Mise à jour inutile dans Langdeck !!

In [ ]:
df_airtable_for_langdeck = get_df_airtable_for_langdeck()
df_airtable_for_langdeck["last_updated"] = df_airtable_for_langdeck[["hash","unixtime"]].apply(lambda x:compare_langdeck_time(x[0],x[1]), axis=1) 
df_newly_updated_in_airtable = df_airtable_for_langdeck.loc[df_airtable_for_langdeck["last_updated"]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  from ipykernel import kernelapp as app


### Récupération des audios

In [ ]:
df_newly_updated_in_airtable


,uid,maitre,ordre,expression,idx,timestamp,language,translation,hash,language_fulltext,unixtime,last_updated
78,8745562b,Comment,3,Comment allez-vous ?,C,2022-04-11 13:41:03,ams,NaN,8745562bams,Arabe,1649684463000000000,True
187,2bd46073,Main,1,Main,M,2022-04-11 13:41:01,ams,NaN,2bd46073ams,Arabe,1649684461000000000,True
201,80e0c323,Médicament,1,Médicament(s),M,2022-04-11 13:41:02,ams,تَورُّم,80e0c323ams,Arabe,1649684462000000000,True
257,ba312208,Âge,3,Quel âge avez-vous ?,A,2022-04-11 13:41:02,ams,ما سبب زيارتك؟,ba312208ams,Arabe,1649684462000000000,True
529,2bd46073,Main,1,Main,M,2022-04-11 13:41:01,eng,Hand,2bd46073eng,Anglais,1649684461000000000,True
...,...,...,...,...,...,...,...,...,...,...,...,...
5045,ba312208,Âge,3,Quel âge avez-vous ?,A,2022-04-11 13:41:02,pol,Ile masz lat ?,ba312208pol,Polonais,1649684462000000000,True
5208,8745562b,Comment,3,Comment allez-vous ?,C,2022-04-11 13:41:03,tig,NaN,8745562btig,Tigrinya,1649684463000000000,True
5317,2bd46073,Main,1,Main,M,2022-04-11 13:41:01,tig,NaN,2bd46073tig,Tigrinya,1649684461000000000,True
5331,80e0c323,Médicament,1,Médicament(s),M,2022-04-11 13:41:02,tig,ከኒና,80e0c323tig,Tigrinya,1649684462000000000,True


## Mise à jour dans Langdeck

In [ ]:
l = len(df_newly_updated_in_airtable)
# on parcoure la table cible
for i in range (2, l):
  # un enregistrement
  tuple_list = t_corpus.row_values(i)
  # on recherche la clé dans le dataframe source
  t_ = df_airtable_for_langdeck.loc[(df_airtable_for_langdeck['uid']==tuple_list[0]) & (df_airtable_for_langdeck['language']==tuple_list[6])]
  # verification du hash
  if len(t_)==1:
    t_corpus.update_cell(i, 9, t_["hash"].values[0])
  # comparaison des ts (A=AT;B=GG)
  ts_a = pd.to_datetime(t_["timestamp"].values[0], "%Y-%m-%d %H:%M:%S")
  ts_b = datetime.datetime.strptime(tuple_list[5], "%Y-%m-%d %H:%M:%S")
  # print (t_["hash"].values[0] + ": Airtable ts " + str(ts_a) + " GG ts " + str(ts_b))
  if ts_a > ts_b:
    # cas Airtable + récent => update dans Langdeck
    t_corpus.update_cell(i, 2, t_["maitre"].values[0])
    t_corpus.update_cell(i, 3, int(t_["ordre"].values[0]))
    t_corpus.update_cell(i, 4, t_["expression"].values[0])
    t_corpus.update_cell(i, 5, t_["idx"].values[0])
    if not pd.isnull(t_["translation"].values[0]):
      t_corpus.update_cell(i, 8, t_["translation"].values[0])    
  elif ts_b > ts_a:
    # cas Langdeck + récent => update translation dans AT uniqumeent !
    record = airtable.match('UID', tuple_list[0])
    if bool(record) is True:
      if not pd.isnull(tuple_list[7]):
        aDict = {}
        aDict[tuple_list[6]] = tuple_list[7]
        airtable.update(record['id'], aDict)
        print (record['id'])


    


In [ ]:
frames = [df_airtable_for_langdeck, df_langdeck]
result = pd.concat(frames, keys=["hash"])
result.drop_duplicates(subset=['hash'], inplace=True, keep='last')

In [ ]:
result

uid      maitre  ordre                                         expression idx           timestamp language                                        translation         hash language_fulltext
hash 0     f9bc13ef  Médicament      3  À quel moment de la journée vous prenez le méd...   M 2022-04-01 08:15:21      ams                  في أي وقت من اليوم تتناول الدواء؟  f9bc13efams             Arabe
     1     6e65dde7       Abcès      1                                              Abcès   A 2022-03-29 15:58:04      ams                                             خُرّاج  6e65dde7ams             Arabe
     2     80cbacea     Abdomen      1                                            Abdomen   A 2022-04-07 08:42:16      ams                                              البطن  80cbaceaams             Arabe
     3     95c4a863     Fébrile      3  Accès fébrile concomitant chez un autre membre...   F 2022-04-01 08:27:19      ams              نوبة حمى مزامنة عند فرد اخر من الأسرة  95c4a863ams             Arabe
     4     30579682    Accident      1                                           Accident   A 2022-03-29 15:58:05      ams                                               حادث  30579682ams             Arabe
...             ...         ...    ...                                                ...  ..                 ...      ...                                                ...          ...               ...
     5451  d112f379       Poids      3                        Votre poids est-il stable ?   P 2022-03-31 14:33:52      tig                                                NaN  d112f379tig          Tigrinya
     5452  2344ba84   Accoucher      3  Vous avez accouché par césarienne ou voie basse ?   A 2022-03-29 16:00:33      tig                             ብኦፕረሽን ወይ ብኖርማር ወሊድኪ ?  2344ba84tig          Tigrinya
     5453  36e66161  Contagieux      3            Y a-t-il des cas contagieux à l'école ?   C 2022-03-29 16:00:33      tig                                 አብ ትምህርቲ ዲዩ ለጊብካ ?  36e66161tig          Tigrinya
     5454  c32f5fd9   Chronique      3  Y a-t-il des maladies chroniques dans votre fa...   C 2022-04-01 08:23:46      tig                      ካብ በተሰብካ ሕዱር ሕማሕ ዘለዎ አለካ ዲዩ ?  c32f5fd9tig          Tigrinya
     5455  d3dba8e4        Zona      1                                               Zona   Z 2022-03-29 16:00:34      tig  ዞና ዝብሃል ሕማም ኣብ ቖርበት ማለት ብደገ ዝቖስል ብርቱዕ ቓንዛ ዘለዎ ...  d3dba8e4tig          Tigrinya

[5456 rows x 10 columns]

In [ ]:
df_airtable_for_langdeck[(df_airtable_for_langdeck['hash'].isin(df_langdeck['hash']) & ~df_airtable_for_langdeck['timestamp'].isin(df_langdeck['timestamp']) )]

,uid,maitre,ordre,expression,idx,timestamp,language,translation,hash,language_fulltext
2,80cbacea,Abdomen,1,Abdomen,A,2022-04-07 08:42:16,ams,البطن,80cbaceaams,Arabe


In [ ]:
set_with_dataframe(t_corpus, result)

END

### Vocabulaire avec groupement :


1.   Index alpha
2.   Mot clé
3.   Ordre



In [ ]:
vk_voc = airtable.get_all(view='Vocabulary', sort=['idx_cor','Maitre','Ordre'])
df_voc = pd.DataFrame.from_records((r['fields'] for r in vk_voc))
df_voc = df_voc[['idx_cor','Maitre','Ordre','Proposition','CATEGRAM','TYPE','Thématique']]

### Traductions avec groupement :


1.   Index alpha
2.   Mot clé
3.   Ordre

In [ ]:
df_voc_trad = airtable.get_all(view='Traductions', sort=['idx_cor','Maitre','Ordre'])
df_voc_trad = pd.DataFrame.from_records((r['fields'] for r in df_voc_trad))
df_voc_trad = df_voc_trad[['idx_cor','Maitre','Ordre','Proposition','CATEGRAM','TYPE','Thématique',
             'eng','ams','tur','rus','ukr','rou','hun','tig','alb','geo','arm',
             'dar','pst','esp','pol']]

In [ ]:
# Corpus_consolidation
uri = "https://docs.google.com/spreadsheets/d/1cSnSnnQunL-I7hpyYkIqbtuGr55unVJ3Qe8URSrjTr8/edit?usp=sharing"

wb = get_CCDB_wb(uri)
db_cc = get_CCDB_data (wb, 0)

nom_onglet = 'Lexique'
# si l'onglet existe déjà
try:
  ws = wb.worksheet(nom_onglet)
  wb.del_worksheet(ws)
except:
  print ("Onglet inexistant !")

wb.add_worksheet(nom_onglet, 1, 1)
export_sheet = wb.worksheet(nom_onglet)
set_with_dataframe(export_sheet, df_voc_trad)

Onglet inexistant !


In [ ]:
grouped = df_voc.groupby(['idx_cor','Maitre'])
'''
for name, group in grouped:
  print (name)
  print (group["Proposition"])
'''

'\nfor name, group in grouped:\n  print (name)\n  print (group["Proposition"])\n'

Recherche des UID manquants dans Airtable

In [ ]:
def airtable_get_missing_uids(df):
  return df.loc[pd.isna(df["UID"])]


Recherche dans Airtable par Séquence

In [ ]:
def airtable_get_records_by_sequence(df, sequence):
  return df.loc[df["Séquence"]==sequence]


In [ ]:
#df_vkat.loc[df_vkat["Séquence"]=="1. Présentation + antécédents"]

In [ ]:
airtable_get_missing_uids(df_vkat)

,Proposition,CATEGRAM,DEPENDANCE,idx_cor,TYPE,UID,VOCABULAIRE,Thématique,idx,Ordre,...,Updates,Séquence,MedGen,Statut,Fiche,Phrase Assoc,Définition,Corpus copy,DOMAINE,Exp Assoc
6,~ chirurgicale,[exp],[reco8iFRctnJgMJCl],I,expression,NaN,NaN,[Vaccination],I,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,Accouchement,[nm],NaN,A,vocabulaire,NaN,NaN,[Médecine générale],NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,Accoucher,[vi],NaN,A,vocabulaire,NaN,NaN,[Médecine générale],NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,Appareil ~,[exp],NaN,D,expression,NaN,NaN,[Médecine générale],NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,Assurance,[nf],NaN,A,vocabulaire,NaN,Général,[Médecine générale],NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,Urine,[nf],NaN,U,vocabulaire,NaN,NaN,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306,Vaccinal,[adj],NaN,V,vocabulaire,NaN,NaN,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308,"Vacciné, -e",[adj],NaN,V,vocabulaire,NaN,NaN,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
317,Votre poids a-t-il augmenté ?,[phrase],[recnFsFSoE1DCjYqt],P,phrase,NaN,NaN,[Médecine générale],NaN,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Rceherche des termes présents dans Central, présents dans AIRTBL mais sans UID

In [ ]:
vk_at_trad = airtable.get_all(view='Traductions',sort='Proposition')
df_vk_at_trad = pd.DataFrame.from_records((r['fields'] for r in vk_at_trad))
df_vk_at_trad = df_vk_at_trad[["Proposition","idx_cor","UID"]]
df_vk_at_trad_notna = df_vk_at_trad.loc[pd.notna(df_vk_at_trad["UID"])]
df_vk_at_trad_na = df_vk_at_trad.loc[pd.isna(df_vk_at_trad["UID"])]

### merge côté AT : avec les UID présents 

In [ ]:
df_merged = pd.merge(df_cc_global, df_vk_at_trad_notna, how="right", left_on='uid', right_on='UID')

## On complète avec les traductions

In [ ]:
for index, row in df_merged.iterrows():
  if row["uid"] is not None:
    record = airtable.match('UID', row["uid"])
    aDict = {}
    for i in vk_languages:
      key = i["trigramme"]
      aDict[key] = row[key]
      #print (record['id'] +" " + row["eng"])
    airtable.update(record['id'], aDict)


## On complète avec les UID manquants

In [ ]:
df_vk_at_trad_na

In [ ]:
df_merged = pd.merge(df_cc_global, df_vk_at_trad_na, how="right", left_on='expression', right_on='Proposition')

In [ ]:
df_merged = df_merged.loc[pd.notna(df_merged["uid"])]

In [ ]:
for index, row in df_merged.iterrows():
  if row["uid"] is not None:
    record = airtable.match('Proposition', row["expression"])
    aDict = {}
    key = "UID"
    aDict[key] = row["uid"]
    for i in vk_languages:
      key = i["trigramme"]
      aDict[key] = row[key]
      #print (record['id'] +" " + row["eng"])
    airtable.update(record['id'], aDict)

## On vide toute la base AT dans Central Consolidation

In [ ]:
vk_at = airtable.get_all(view='Admin',sort='Proposition')
df_vkat = pd.DataFrame.from_records((r['fields'] for r in vk_at))

In [ ]:
# Corpus_consolidation
uri = "https://docs.google.com/spreadsheets/d/1cSnSnnQunL-I7hpyYkIqbtuGr55unVJ3Qe8URSrjTr8/edit?usp=sharing"

wb = get_CCDB_wb(uri)
db_cc = get_CCDB_data (wb, 0)

nom_onglet = 'Termes'
# si l'onglet existe déjà
try:
  ws = wb.worksheet(nom_onglet)
  wb.del_worksheet(ws)
except:
  print ("Onglet inexistant !")

wb.add_worksheet(nom_onglet, 1, 1)
export_sheet = wb.worksheet(nom_onglet)
set_with_dataframe(export_sheet, df_vkat)



END

In [ ]:
record = airtable.match('UID', '30579682')
fields = {'eng': 'Accident'}
airtable.update(record['id'], fields)

{'createdTime': '2021-12-10T08:47:48.000Z',
 'fields': {'CATEGRAM': ['nm'],
  'Name ID': 'recGaRQGjV5qN3QBN',
  'Proposition': 'Accident',
  'TYPE': 'vocabulaire',
  'Thématique': 'Vaccination',
  'UID': '30579682',
  'VOCABULAIRE': 'Général',
  'eng': 'Accident',
  'idx': 'A',
  'idx_cor': 'A'},
 'id': 'recGaRQGjV5qN3QBN'}

In [ ]:
record

{'createdTime': '2021-12-10T08:47:48.000Z',
 'fields': {'CATEGRAM': ['nm'],
  'Name ID': 'recGaRQGjV5qN3QBN',
  'Proposition': 'Accident',
  'TYPE': 'vocabulaire',
  'Thématique': 'Vaccination',
  'UID': '30579682',
  'VOCABULAIRE': 'Général',
  'idx': 'A',
  'idx_cor': 'A'},
 'id': 'recGaRQGjV5qN3QBN'}

In [ ]:
def airtable_table_get(uri, params, headers):
  airtable_records = []
  run = True
  while run is True:
    response = requests.get(uri, params=params, headers=headers)
    airtable_response = response.json()
    airtable_records += (airtable_response['records'])
    if 'offset' in airtable_response:
      run = True
      params = (('offset', airtable_response['offset']),)
    else:
      run = False
  return airtable_records

# Table Corpus

In [ ]:

url = "https://api.airtable.com/v0/" + base_id + "/" + table_name


### On lit tous les enregistrements, puis on met dans un dataframe

In [ ]:
airtable_records = airtable_table_get(url, params, headers)

airtable_rows = [] 
airtable_index = []
for record in airtable_records:
  airtable_rows.append(record['fields'])
  airtable_index.append(record['id'])
df = pd.DataFrame(airtable_rows, index=airtable_index)
df = df [['Proposition','UID', 'Name ID', 'DEPENDANCE', 'idx', 'idx_cor','Statut','Thématique']]
df.sort_values(by=['Proposition'], ascending=True, inplace=True)

In [ ]:
df.head(10)

,Proposition,UID,Name ID,DEPENDANCE,idx,idx_cor,Statut,Thématique
recKbPHqDTXNPwYK0,A jeun,f397cda1,recKbPHqDTXNPwYK0,NaN,J,J,NaN,Vaccination
rec1qTnfWxjhx6SBl,AME - Aide médicale d'État,f49563a9,rec1qTnfWxjhx6SBl,NaN,A,A,NaN,Vaccination
rec0HgtD22F7TWM3W,Abcès,6e65dde7,rec0HgtD22F7TWM3W,NaN,A,A,NaN,Médecine générale
recGaRQGjV5qN3QBN,Accident,30579682,recGaRQGjV5qN3QBN,NaN,A,A,NaN,Vaccination
rec3MxDq67MC982ng,Accident cérébral,6f214e4c,rec3MxDq67MC982ng,[recGaRQGjV5qN3QBN],A,A,NaN,Vaccination
reciexfGbNKvXzQg8,Alcool,NaN,reciexfGbNKvXzQg8,NaN,A,A,NaN,Médecine générale
recfY4uKkgK8Jp8GO,Allergie,8b195cf4,recfY4uKkgK8Jp8GO,NaN,A,A,NaN,Vaccination
recSKSrXVLhDfMq6w,Allergie alimentaire,f754eebf,recSKSrXVLhDfMq6w,[recfY4uKkgK8Jp8GO],A,A,NaN,Vaccination
rectthHdPchXi1zr9,Allergie médicamenteuse,7256c169,rectthHdPchXi1zr9,"[recfY4uKkgK8Jp8GO, recOivQOmeragygVr]",A,A,NaN,Vaccination
recHNzXfvWWm0Lwp8,Allergie à un médicament,316fa7ab,recHNzXfvWWm0Lwp8,"[recfY4uKkgK8Jp8GO, recOivQOmeragygVr]",A,A,NaN,Vaccination


In [ ]:
def lookup_uid (uid):
  try:
    record = df_cc_fr.loc[df_cc_fr["uid"]==uid]["index"].item()
  except Exception as e:
    list_no_uid.append(uid)
    return uid
  if record is not None:
    return record

In [ ]:
x = df_cc_fr.loc[df_cc_fr["uid"]=="72f0e8db"]["index"].item()

In [ ]:
list_no_uid = [] # uid non présents côté central = mots à créer
df['UID'] = df['UID'].apply(lambda x: make_unique_id() if pd.isna(x) else x)
df.idx_cor = df['UID'].apply(lambda x: lookup_uid(x))

convert the dataframe to a dictionary of dictionaries.

In [ ]:
df.tail()

,Proposition,UID,Name ID,DEPENDANCE,idx,idx_cor,Statut,Thématique
recia1L6nICYg6LJ8,Zona,d3dba8e4,recia1L6nICYg6LJ8,NaN,NaN,Z,NaN,Vaccination
recVvIHdNWf80zvsG,À quelle moment de la journée vous prenez le m...,0f86d1ef,recVvIHdNWf80zvsG,[recOivQOmeragygVr],M,0f86d1ef,Ramona,Médecine générale
recWp0rajm2qKcJuF,Êtes vous vacciné (e) ?,c5df267a,recWp0rajm2qKcJuF,NaN,NaN,Ê,NaN,Médecine générale
recBY5NXGM8UtBcaX,Êtes-vous enceinte ?,7768882c,recBY5NXGM8UtBcaX,NaN,NaN,Ê,NaN,Médecine générale
reczy8Os5Hc3qZ1yA,Œdème,72f0e8db,reczy8Os5Hc3qZ1yA,NaN,NaN,Œ,NaN,Médecine générale


In [ ]:
dict_records = df.tail().to_dict(orient="index")

In [ ]:
for record_id, upload_data in dict_records.items():
  record_url = url + "/" + record_id
  upload_dict = {"records" : [{"fields" : upload_data}]}
  upload_json = json.dumps(upload_dict)
  requests.post(url, data=upload_json, headers=headers)

In [ ]:
headers

{'Authorization': 'Bearer keyrI98TIqu6mbFcf',
 'Content-Type': 'application/json'}

In [ ]:
requests.patch(url, data=upload_json, headers=headers)

<Response [422]>

In [ ]:
upload_json='{"records": [{"fields": {"Proposition": "OEDEME", "id": "reczy8Os5Hc3qZ1yA"}}]}'

In [ ]:
requests.post(record_url, data=upload_json, headers=headers)

<Response [404]>

In [ ]:
def fmt_table(wb, nom_onglet):
  # onglet
  ws = wb.worksheet(nom_onglet)

  # Format de la partie gauche
  fmt = cellFormat(
      backgroundColor=color(0.91, 0.96, 0.93),
      textFormat=textFormat(bold=False, foregroundColor=color(0,0,0), fontSize='10'),
      horizontalAlignment='LEFT'
      )
  format_cell_range(ws, 'A:', fmt)

  # format de l'entete
  fmt = cellFormat(
      backgroundColor=color(0.7725,0.8431,0.7922),
      textFormat=textFormat(bold=True, foregroundColor=color(0,0,0), fontSize='10'),
      horizontalAlignment='LEFT'
      )
  format_cell_range(ws, '1', fmt)
  
  # format de l'auto wrap
  fmt = cellFormat(
      wrapStrategy='WRAP'
      )
  format_cell_range(ws, 'B:C', fmt)

  # on gèle l'entete et les colonnes de gauche
  set_frozen(ws, rows=1, cols=1)
  set_column_width(ws, 'A:C', 500)


In [ ]:
nom_onglet = 'Liste'
# si l'onglet existe déjà
try:
  ws = wb.worksheet(nom_onglet)
  wb.del_worksheet(ws)
except:
  print ("Onglet inexistant !")

wb.add_worksheet(nom_onglet, 1, 1)
export_sheet = wb.worksheet(nom_onglet)
set_with_dataframe(export_sheet, df)
fmt_table(wb,nom_onglet)


In [ ]:
table_name = 'Structure'
url = "https://api.airtable.com/v0/" + base_id + "/" + table_name

In [ ]:
df

,Name,Situation de départ,Notes
recnI80hvydv71fs4,0.Conception,"Plus on possède, plus on est soi-même possédé....","Trouver le plaisir d’écrire. Partir d’idées, d..."
recI2pgkNq2iP52rm,1.Situation initiale,Un jour d'été où il pleut et fait un temps abo...,Une vengeance d'une femme éconduite lors d'un ...
recb2DqhB9GvfI12n,2.Personnage,Un homme (marié ?) qui a donné rendez-vous à u...,Un Dom Juan sûr de son charme et imbu de sa pe...
recNLdU01JpCnvknJ,3.Acte déclencheur,La femme lui propose un jeu. Elle est dans le ...,NaN
recRxZVhpSPD6oQXx,4.Réaction,L'homme est d'abord tenté de partir car il est...,NaN
recRr06fOM4hGGaLY,5.Péripéties,"Pendant la visite (ennuyeuse), l'homme tente d...",NaN
recvWnAYn9VNySVlB,6.Chute,Un spectacle où une partie du public est appel...,NaN
recDmZtMMNmiy7R0M,Récit,NaN,NaN
